In [8]:
codex_tags = {
}

from codex_widget_factory import utils
results_json=[]

In [1]:
##

In [9]:

step1 = """

import plotly.express as px
import pandas as pd
import json
from io import StringIO
import plotly.io as io


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def file_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        def read_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=read_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_graph(dframe, filters):
    logger.info(
        "Preparing bar graph json to understand basic_expenditure across all countries")
    column_names = ['x48', 'x38', 'x37', 'x36', 'x33', 'x8', 'x29']
    #print(dframe['country'])
    data_good = dframe[dframe["data_quality"] == 1]
    data_good['basic_expenditure']= data_good[column_names].sum(axis=1)
    for item in filters:
        if 'All' in filters[item]:
            continue
        elif isinstance(filters[item], list):
            data_good = data_good[data_good[item].isin(filters[item])]
        else:
            data_good = data_good[data_good[item] == filters[item]]
    fig = px.bar(data_good, x='x48', y='country', color='basic_expenditure')
    #fig.show()
    logger.info(
        "Successfully prepared bar graph json to understand basic_expenditure across all countries")
    return io.to_json(fig)


selected_filters = {"country": ['India','China','Russia']}
dframe = file_read("cost-of-living_v2_I0869.csv")
#dframe = dframe.groupby(['country'])
dynamic_outputs = get_graph(dframe, selected_filters)#here11

"""
# #END CUSTOM CODE

In [17]:
#step2_Table = """
# Below codestring is used to display the grid table that consists of data of cost of basic ammenities.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def file_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        # input_df = input_df[input_df.x2.notnull()]
        # input_df = input_df[input_df.x23.notnull()]
        # input_df = input_df[input_df.x33.notnull()]
        # input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(data_good)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_filter_table(dframe, selected_filters):
    logger.info("Applying screen filters on the grid table dframe.")
    select_df = dframe.copy()
    for item in list(selected_filters):
        if isinstance(selected_filters[item], list):
            if 'All' not in selected_filters[item] and selected_filters[item]:
                select_df = select_df[select_df[item].isin(
                    selected_filters[item])]
        else:
            if selected_filters[item] != 'All':
                select_df = select_df[select_df[item]
                                      == selected_filters[item]]
    logger.info("Successfully applied screen filters on the grid table dframe.")
    return select_df


def generate_dynamic_table(dframe, name='Sales', grid_options={"tableSize": "small", "tableMaxHeight": "80vh", "quickSearch":True}, group_headers=[], grid="auto"):
    logger.info("Generate dynamic Grid table json from dframe")
    table_dict = {}
    table_props = {}
    table_dict.update({"grid": grid, "type": "tabularForm",
                      "noGutterBottom": True, 'name': name})
    values_dict = dframe.dropna(axis=1).to_dict("records")
    table_dict.update({"value": values_dict})
    col_def_list = []
    for col in list(dframe.columns):
        col_def_dict = {}
        col_def_dict.update({"headerName": col, "field": col})
        col_def_list.append(col_def_dict)
    table_props["groupHeaders"] = group_headers
    table_props["coldef"] = col_def_list
    table_props["gridOptions"] = grid_options
    table_dict.update({"tableprops": table_props})
    logger.info("Successfully generated dynamic Grid table json from dframe")
    return table_dict


def build_grid_table_json():
    logger.info("Preparing grid table json for Table Screen")
    form_config = {}
    dframe = file_read("cost-of-living_v2_I0869.csv")

    dframe=dframe[['country','city','x48', 'x41', 'x36', 'x8','x30']]
    dframe.columns = dframe.columns.str.replace('x48', 'Apartment')
    dframe.columns = dframe.columns.str.replace('x41', 'Movie Ticket')
    dframe.columns = dframe.columns.str.replace('x36', 'Basic cost')
    dframe.columns = dframe.columns.str.replace('x8', 'Water')
    dframe.columns = dframe.columns.str.replace('x30', 'Taxi')
    #dframe=dframe.sample(n = 50)
    #print(dframe)
    
    selected_filters = {"country": ['India','China','Russia','United States']}

    dframe2 = get_filter_table(dframe, selected_filters)
    dframe2=dframe2.sample(n = 50)
    print(dframe2)
    form_config['fields'] = [generate_dynamic_table(dframe2)]
    grid_table_json = {}
    grid_table_json['form_config'] = form_config
    logger.info("Successfully prepared grid table json for Table Screen")
    print(grid_table_json)
    return grid_table_json


grid_table_json = build_grid_table_json()
dynamic_outputs = json.dumps(grid_table_json)

#"""

            country             city  Apartment  Movie Ticket  Basic cost  \
632   United States          El Paso     916.67         11.00      193.29   
299   United States         Columbus    1394.44         12.00      204.48   
1599  United States           Tacoma    1775.00         12.50      192.59   
1322  United States            Plano    1666.67         12.00      238.65   
1487  United States          Fremont    2312.50         14.50      247.78   
601   United States       Fort Worth    1387.50         12.50      212.50   
948   United States       Long Beach    2175.00         15.00      159.71   
142   United States          Seattle    2291.82         15.00      221.73   
1804  United States  Fort Lauderdale    2381.25         13.00      138.95   
225   United States        Baltimore    1559.96         12.00      148.18   
374          Russia             Omsk     325.33          4.00      126.82   
1151  United States       Greensboro    1400.00         12.00      162.37   

In [18]:
dynamic_outputs

'{"form_config": {"fields": [{"grid": "auto", "type": "tabularForm", "noGutterBottom": true, "name": "Sales", "value": [{"country": "United States", "city": "El Paso", "Apartment": 916.67, "Movie Ticket": 11.0, "Basic cost": 193.29, "Water": 1.19, "Taxi": 1.65}, {"country": "United States", "city": "Columbus", "Apartment": 1394.44, "Movie Ticket": 12.0, "Basic cost": 204.48, "Water": 1.59, "Taxi": 3.5}, {"country": "United States", "city": "Tacoma", "Apartment": 1775.0, "Movie Ticket": 12.5, "Basic cost": 192.59, "Water": 1.81, "Taxi": 3.5}, {"country": "United States", "city": "Plano", "Apartment": 1666.67, "Movie Ticket": 12.0, "Basic cost": 238.65, "Water": 1.78, "Taxi": 2.95}, {"country": "United States", "city": "Fremont", "Apartment": 2312.5, "Movie Ticket": 14.5, "Basic cost": 247.78, "Water": 2.29, "Taxi": 3.0}, {"country": "United States", "city": "Fort Worth", "Apartment": 1387.5, "Movie Ticket": 12.5, "Basic cost": 212.5, "Water": 1.62, "Taxi": 2.25}, {"country": "United Sta

In [11]:
step3_Table = """
# Below codestring is used to display the grid table that consists of historic sales done over time on each product.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def file_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869' file_read("cost-of-living_v2_I0869")
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x8.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        input_df = input_df[input_df.x48.notnull()]
        input_df = input_df[input_df.x37.notnull()]
        input_df = input_df[input_df.x36.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        #print(data_good)
        return(data_good)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_filter_table(dframe, selected_filters):
    logger.info("Applying screen filters on the grid table dframe.")
    select_df = dframe.copy()
    for item in list(selected_filters):
        if isinstance(selected_filters[item], list):
            if 'All' not in selected_filters[item] and selected_filters[item]:
                select_df = select_df[select_df[item].isin(
                    selected_filters[item])]
        else:
            if selected_filters[item] != 'All':
                select_df = select_df[select_df[item]
                                      == selected_filters[item]]
    logger.info("Successfully applied screen filters on the grid table dframe.")
    return select_df


def generate_dynamic_table(dframe, name='Sales', grid_options={"tableSize": "small", "tableMaxHeight": "80vh", "quickSearch":True}, group_headers=[], grid="auto"):
    logger.info("Generate dynamic Grid table json from dframe")
    table_dict = {}
    table_props = {}
    table_dict.update({"grid": grid, "type": "tabularForm",
                      "noGutterBottom": True, 'name': name})
    values_dict = dframe.dropna(axis=1).to_dict("records")
    table_dict.update({"value": values_dict})
    col_def_list = []
    for col in list(dframe.columns):
        col_def_dict = {}
        col_def_dict.update({"headerName": col, "field": col})
        col_def_list.append(col_def_dict)
    table_props["groupHeaders"] = group_headers
    table_props["coldef"] = col_def_list
    table_props["gridOptions"] = grid_options
    table_dict.update({"tableprops": table_props})
    logger.info("Successfully generated dynamic Grid table json from dframe")
    return table_dict


def build_grid_table_json():
    logger.info("Preparing grid table json for Colour Screen")
    form_config = {}
    dframe = file_read("cost-of-living_v2_I0869.csv")
    dframe=dframe[['country','city','x48', 'x41', 'x36', 'x8']] #--here281222 --5 columns
    dframe.columns = dframe.columns.str.replace('x48', 'Apartment')
    dframe.columns = dframe.columns.str.replace('x41', 'Movie Ticket')
    dframe.columns = dframe.columns.str.replace('x36', 'Basic cost')
    dframe.columns = dframe.columns.str.replace('x8', 'Water')
    
    selected_filters = {"country": ['India','United States']}
    
    dframe2 = get_filter_table(dframe, selected_filters)
    form_config['fields'] = [generate_dynamic_table(dframe2)]
    grid_table_json = {}
    grid_table_json['form_config'] = form_config
    logger.info("Successfully prepared grid table json for Colour Screen")
    return grid_table_json

grid_table_json = build_grid_table_json()
dynamic_outputs = json.dumps(grid_table_json)

# """


In [12]:
step4_a ="""
import numpy as np
import pandas as pd
import plotly.express as px
import pandas as pd
import json
from io import StringIO
import plotly.io as io
from plotly.io import to_json




def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()



def getGraph():
    logger.info(
        "Preparing scatter plot json to understand relation between x and y values")
    df = pd.DataFrame(np.random.randint(0,100,size=(100, 1)), columns=list('X'))
    c = 4
    df = df.assign(Y = lambda x: (x['X']**3+c))
    fig = px.scatter(df, y='Y', x='X')
    fig.show()
    return io.to_json(fig)
dynamic_outputs = getGraph()
#print(df)
"""

In [13]:
step5 ="""
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import json
from io import StringIO
import plotly.io as io
from plotly.io import to_json



def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()

def file_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")

def getGraph():
    logger.info(
        "Preparing scatter plot json to understand relation between x and y values")
    # load dataset
    #df = pd.DataFrame(np.random.randint(0,100,size=(100, 2)), columns=list('XY'))
    #df = df.assign(Y = lambda x: (x['X']**2))
    #df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/volcano.csv")
    input_df=file_read("cost-of-living_v2_I0869.csv")
        #filtering out empty columns
    df = input_df[input_df["data_quality"] == 1].groupby(['city'])
    df = input_df[input_df["country"] == 'India']
    #print(df)
    # create figure
    fig = go.Figure()

    # Add surface trace
    fig.add_trace(go.Surface(  x=df.city,y=df.x54, colorscale="Viridis"))

    # Update plot sizing
    # fig.update_layout(
    #     width=800,
    #     height=900,
    #     autosize=False,
    #     margin=dict(t=0, b=0, l=0, r=0),
    #     template="plotly_white",
    # )

    # Update 3D scene options
    # fig.update_scenes(
    #     aspectratio=dict(x=1, y=1, z=0.7),
    #     aspectmode="manual"
    # )

    # Add dropdown
    fig.update_layout(
        updatemenus=[
            dict(
                buttons=list([
                    dict(
                        args=["type", "bar"],
                        label="bar",
                        method="restyle"
                    ),
                    dict(
                        args=["type", "scatter"],
                        label="scatter",
                        method="restyle"
                    ),
                    dict(
                        args=["type", "violin"],
                        label="violin",
                        method="restyle"
                    ),
                    dict(
                        args=["type", "column"],
                        label="column",
                        method="restyle"
                    ),
                    dict(
                        args=["type", "box"],
                        label="box",
                        method="restyle"
                    )
                ]),
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    # Add annotation
    fig.update_layout(
        annotations=[
            dict(text="Trace type:", showarrow=False,
            x=0, y=1.085, yref="paper", align="left")
        ]
    )

    fig.show()
    return io.to_json(fig)

dynamic_outputs = getGraph()
"""

In [14]:
plots_dict={
    "step_2_Table":step2_Table,
    "step_3_Table":step3_Table,
    "step_1":step1,
    "step_4_a":step4_a,
    "step_5":step5
}
results_json.append({
    'type':'review',
    'name': 'overview',
    'component':'overview',
    'dynamic_visual_results': plots_dict      
})




In [15]:

currentNotebook = 'cost_forecasting_I01176_202212281524.ipynb'

!jupyter nbconvert --to script {currentNotebook} 



(4, 2) {}
{'$schema': 'http://json-schema.org/draft-04/schema#', 'description': 'Jupyter Notebook v4.2 JSON schema.', 'type': 'object', 'additionalProperties': False, 'required': ['metadata', 'nbformat_minor', 'nbformat', 'cells'], 'properties': {'metadata': {'description': 'Notebook root-level metadata.', 'type': 'object', 'additionalProperties': True, 'properties': {'kernelspec': {'description': 'Kernel information.', 'type': 'object', 'required': ['name', 'display_name'], 'properties': {'name': {'description': 'Name of the kernel specification.', 'type': 'string'}, 'display_name': {'description': 'Name to display in UI.', 'type': 'string'}}}, 'language_info': {'description': 'Kernel information.', 'type': 'object', 'required': ['name'], 'properties': {'name': {'description': 'The programming language which this kernel runs.', 'type': 'string'}, 'codemirror_mode': {'description': 'The codemirror mode to use for code in this language.', 'oneOf': [{'type': 'string'}, {'type': 'object'}

[NbConvertApp] Converting notebook cost_forecasting_I01176_202212281524.ipynb to script
[NbConvertApp] Writing 18899 bytes to cost_forecasting_I01176_202212281524.py


In [16]:

utils.submit_config_params(url='https://codex-api-stage.azurewebsites.net/codex-api/projects/upload-config-params/vRhoxvpFGK9b_LkarsEvWQ', nb_name=currentNotebook, results=results_json, codex_tags=codex_tags, args={})

['codex_widget_factory']
SUCCESS | Submitted config params.
